# Needed libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import tqdm
import time
from scipy import stats
import seaborn as sbn
import pandas as pd
import h5py
import os

In [2]:
from keras.layers import Lambda, Input, Dense, Dropout, AveragePooling2D, Flatten, Conv2D
from keras.layers import MaxPooling2D, BatchNormalization, Activation
from keras.models import Model, Sequential
from keras.losses import mse, binary_crossentropy, mean_absolute_percentage_error
from keras.utils import plot_model,to_categorical
from keras import backend as K
from keras import callbacks
#from sklearn.preprocessing import LabelEncoder,OneHotEncoder,image,label_binarize
from keras.utils import plot_model
from keras.optimizers import SGD, Adam
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from sklearn.metrics import roc_curve,roc_auc_score,mean_squared_error
#from keras.applications.resnet50 import ResNet50
import tensorflow as tf
import keras.losses as kLoss
from keras import optimizers

2025-02-17 15:49:26.879391: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
print('numpy version:', np.__version__)
print('matplotlib version:', mpl.__version__)
print('tensorflow version:', tf.__version__)

color_rate = "#d55e00"
color_drate = 'darkblue' #"#0072b2"
color_s1s2 = 'limegreen' #"#009e73"
color_comb = 'limegreen'

# Check if gpu is available
if tf.test.is_gpu_available():
    device = 'gpu'
    print('Using GPU')
else:
    device = 'cpu'
    print('Using CPU')

numpy version: 1.22.4
matplotlib version: 3.5.2
tensorflow version: 2.9.1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Using GPU


2025-02-17 15:49:36.475514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-17 15:49:36.612947: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:49:36.819681: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:49:36.820155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so ret

# Reading data

In [4]:
images = np.load('../data/TNGgalaxies/images.npy')
labels = np.load('../data/TNGgalaxies/labels.npy')

In [5]:
print(f'There are {len(np.where(np.isnan(images) == True)[0])} nans in images')
print(f'There are {len(np.where(np.isnan(labels) == True)[0])} nans in labels')

There are 4444 nans in images
There are 0 nans in labels


In [6]:
images = np.nan_to_num(images, nan=0)

In [7]:
images = np.moveaxis(images, 1, -1)

In [8]:
nobs, npix, _, nch = images.shape

In [9]:
np.random.seed(28890)
random_ind = np.random.choice(np.arange(nobs), size = nobs, replace = False)

train_size = int(0.7 * nobs)
val_size = int(0.2 * nobs)
test_size = nobs - train_size - val_size

train_ind = random_ind[:train_size]
val_ind = random_ind[train_size:(val_size + train_size)]
test_ind = random_ind[-test_size:]

In [10]:
x_trainset = images[train_ind,:,:,:]
x_valset = images[val_ind,:,:,:]
x_testset = images[test_ind,:,:,:]

y_trainset = np.log10(labels[train_ind,0,:])
y_valset = np.log10(labels[val_ind,0,:])
y_testset = np.log10(labels[test_ind,0,:])

In [11]:
min_x = np.min(x_trainset)
max_x = np.max(x_trainset)

min_y = np.min(y_trainset, axis = 0)
max_y = np.max(y_trainset, axis = 0)

In [14]:
x_trainset = (x_trainset - min_x) / (max_x - min_x)
x_valset = (x_valset - min_x) / (max_x - min_x)
x_testset = (x_testset - min_x) / (max_x - min_x)

y_trainset = (y_trainset - min_y) / (max_y - min_y)
y_valset = (y_valset - min_y) / (max_y - min_y)
y_testset = (y_testset - min_y) / (max_y - min_y)

In [13]:
min_x

-1218547.7897511267

# Playing with CNNs

In [15]:
# network parameters
input_shape = (npix, npix, nch) # Input shape (#rows, #cols, #channels)
actFunction = 'relu'

# Hidden layers dimensions
intermediate_dim1 = 500
intermediate_dim2 = 400
intermediate_dim3 = 300
intermediate_dim4 = 200
output_dim        = y_trainset.shape[1]

In [16]:
# build model
model = Sequential()

model.add(Conv2D(filters = 5, kernel_size = (5,5), strides = 1 , padding = 'same', input_shape = input_shape))
model.add(BatchNormalization())
model.add(Activation(actFunction))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 15, kernel_size = (5,5), strides = 1 , padding = 'same'))
model.add(BatchNormalization())
model.add(Activation(actFunction))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters = 30, kernel_size = (5,5), strides = 1 , padding = 'same'))
model.add(BatchNormalization())
model.add(Activation(actFunction))
model.add(MaxPooling2D(pool_size=(2, 2)))

#model.add(Dropout(0.2)) 
model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(intermediate_dim1, activation = actFunction))

model.add(BatchNormalization())
model.add(Dense(intermediate_dim2, activation = actFunction))

model.add(BatchNormalization())
model.add(Dense(intermediate_dim3, activation = actFunction))

model.add(BatchNormalization())
model.add(Dense(intermediate_dim4, activation = 'linear'))

model.add(Dense(output_dim, name = 'output'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 5)       1005      
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 5)      20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 128, 5)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 5)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 15)        1890      
                                                                 
 batch_normalization_1 (Batc  (None, 64, 64, 15)       6

2025-02-17 15:50:45.533427: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:50:45.533977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:50:45.534363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:50:45.535308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-17 15:50:45.535717: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [ ]:
#gen = ImageDataGenerator(rotation_range = 90) # Degree range for random rotations.

In [17]:
# instantiate model
optimizer = optimizers.Adam(learning_rate = 1e-5, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
model.compile(optimizer = optimizer, loss = 'mse', metrics=['mae','mse'])

In [18]:
# This are some callbacks to take control of the NN
#es = callbacks.EarlyStopping(monitor = 'val_loss', mode = min, verbose = 1, patience = 100) # Stop after "patience" number of steps without improving
mc = callbacks.ModelCheckpoint('../data/best_model.h5', monitor ='val_loss', mode = 'min', verbose = 1, save_best_only = True) # Save only the best epoch

In [19]:
images.shape

(2972, 128, 128, 8)

In [ ]:
model(images[20:21,:,:,:])

In [67]:
batch_size = 32
epochs     = 25
history = model.fit(x_trainset[:batch_size], y_trainset[:batch_size],
                   epochs = epochs,
                   callbacks = [mc],
                   validation_data = (x_trainset[:batch_size], y_trainset[:batch_size]))

Epoch 1/25
1/1 [==============================] - ETA: 0s - loss: nan - mae: nan - mse: nan
Epoch 1: val_loss did not improve from inf
1/1 [==============================] - 1s 685ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 2/25
1/1 [==============================] - ETA: 0s - loss: nan - mae: nan - mse: nan
Epoch 2: val_loss did not improve from inf
1/1 [==============================] - 0s 24ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 3/25
1/1 [==============================] - ETA: 0s - loss: nan - mae: nan - mse: nan
Epoch 3: val_loss did not improve from inf
1/1 [==============================] - 0s 23ms/step - loss: nan - mae: nan - mse: nan - val_loss: nan - val_mae: nan - val_mse: nan
Epoch 4/25
1/1 [==============================] - ETA: 0s - loss: nan - mae: nan - mse: nan
Epoch 4: val_loss did not improve from inf
1/1 [==============================] - 0s 25ms/step - loss: nan

In [ ]:
batch_size = 32
epochs     = 250
#ran_ind    = np.random.choice(range(len(x_trainset)), size = len(x_trainset), replace = True)
#generator  = gen.flow(x_trainset[ran_ind,:,:,:], y_trainset[ran_ind,:], batch_size = batch_size)
generator  = gen.flow(x_trainset, y_trainset, batch_size = batch_size)
# change epochs
history = model.fit_generator(
                  generator,
                  steps_per_epoch  = len(x_trainset)/batch_size,
                  epochs           = epochs,
                  verbose          = 1,
                  validation_data  = (x_valset, y_valset),
                  validation_steps = len(x_valset)/batch_size,
                  callbacks        = [es, mc])